In [1]:
!pip install pytrends


https://pypi.org/project/pytrends/

In [1]:
import json
import math
import time
import pytrends
import pandas as pd
from datetime import datetime
from pytrends.request import TrendReq
from dateutil.relativedelta import relativedelta


In [2]:
pytrends = TrendReq(hl='en-US', tz=360)
#, timeout=(10,25), proxies=['http://172.16.14.3',], retries=2, backoff_factor=0.1, requests_args={'verify':False}

covid_keywords = ['Covid','Covid19', 'Covid-19', 'Coronavirus', 'quarantine', \
                  'isolation', 'Outbreak', 'muscle aches', 'body aches', \
                 'high temperature', 'Fatigue', 'Fever', 'chills', 'loss of smell', \
                  'loss of taste', 'cannot smell',  "can't smell","can't taste", \
                  'cannot taste', 'Shortness of breath', 'difficulty breathing', \
                  "can't breath", "cannot breath", 'Cough', 'Sore throat', 'sneezing', \
                 'casual contact', 'close contact', 'direct contact', 'mild cold symptoms', 'mild cold', \
                  'flu-like symptoms', 'flu-like', \
                 'pandemic', 'epidemic', 'epidemy']

In [3]:
contry_codes_df = pd.read_csv('ISO_contry_codes.csv')[['country', 'ISO-3166_countr_code']]
ISO_codes = list(contry_codes_df['ISO-3166_countr_code'])
contry_codes_df.head()

,country,ISO-3166_countr_code
0,Afghanistan,AF
1,Aland Islands,AX
2,Albania,AL
3,Algeria,DZ
4,American Samoa,AS


In [4]:

def execute(keywords_list, period, stat_type='', geo_code='US'):
    full_stat = pd.DataFrame()
    for i in range(math.floor(len(covid_keywords)/5)):
        print("Building payload with keywords: ", covid_keywords[i*5: i*5 + 5], "\nperiod: ", period)
        full_stat = pd.concat([full_stat, get_stat(covid_keywords[i*5: i*5 + 5], period, stat_type, geo_code)])
  
       
    if (len(covid_keywords)%5):
        print("Building payload with keywords: ", covid_keywords[math.floor(len(covid_keywords)/5)*5 :])
        return pd.concat([full_stat, \
                              pd.DataFrame(get_stat(covid_keywords[math.floor(len(covid_keywords)/5)*5 :], \
                                                    period, stat_type, geo_code))])
    else:
        return full_stat


def get_stat(keywords, period, stat_type, geo_code):
    time.sleep(100)
    pytrends.build_payload(keywords, timeframe=period, geo=geo_code) #geo в формате ISO 3166-1
    if (stat_type == 'by_region'):
        return pytrends.interest_by_region(resolution='COUNTRY', inc_low_vol=True, inc_geo_code=False)
    return pytrends.interest_over_time()


In [ ]:
#Мировая ежедневная статистика covid
covid_stat = execute(covid_keywords, '2019-10-01 2021-04-09')
covid_stat.dropna(how='all', axis=1).fillna(0).to_csv('daily_covid_stat_world.csv')

In [182]:
#ПОдсчет общего количества keyword-поисковых запросов за период пандемии
total_search_cases = {}
for i in covid_stat.drop(columns='isPartial', axis=1).columns:
    total_search_cases[i] = covid_stat[i].sum()
print("Total Google Trends search cases from 2019-10-01 to 2021-04-09:\n", total_search_cases)
    

Total Google Trends search cases from 2019-10-01 to 2021-04-09:
 {'Covid': 944.0, 'Covid19': 17.0, 'Covid-19': 92.0, 'Coronavirus': 858.0, 'quarantine': 41.0, 'isolation': 643.0, 'Outbreak': 1139.0, 'muscle aches': 72.0, 'body aches': 288.0, 'high temperature': 320.0, 'Fatigue': 607.0, 'Fever': 2891.0, 'chills': 293.0, 'loss of smell': 81.0, 'loss of taste': 85.0, 'cannot smell': 3.0, "can't smell": 187.0, "can't taste": 171.0, 'cannot taste': 11.0, 'Shortness of breath': 2370.0, 'difficulty breathing': 71.0, "can't breath": 49.0, 'cannot breath': 0.0, 'Cough': 2901.0, 'Sore throat': 1599.0, 'sneezing': 3749.0, 'casual contact': 7.0, 'close contact': 443.0, 'direct contact': 360.0, 'mild cold symptoms': 11.0, 'mild cold': 0.0, 'flu-like symptoms': 0.0, 'flu-like': 0.0, 'pandemic': 1434.0, 'epidemic': 197.0, 'epidemy': 1474.0}


In [217]:
with open('GT_total_search_cases.csv', "w") as f:
    json.dump(total_search_cases, f)

In [5]:
#статистика по США по месяцам
pandemy_stat = pd.DataFrame()
start_of_period = "2019-10-01"
end_of_period = "2019-11-01"
dt_format = "%Y-%m-%d"

def add_month(date, dt_format=dt_format):
    return datetime.strftime((datetime.strptime(date, dt_format) + relativedelta(months=1)), dt_format)

while datetime.strptime(end_of_period, dt_format) < datetime.now():
    full_period = start_of_period + ' ' + end_of_period
    pandemy_stat[full_period] = execute(covid_keywords, full_period, 'by_region')
    start_of_period, end_of_period = add_month(start_of_period), add_month(end_of_period)
    
monthly_stat_by_country = pd.DataFrame()
for i in pandemy_report.keys():
    item = pandemy_report[i]
    item['period_from'] = datetime.strptime(i[:10], dt_format)
    item['period_to'] = datetime.strptime(i[11:], dt_format)
    monthly_stat_by_country = pd.concat([monthly_stat_by_country, item])
    
monthly_stat_by_country.dropna(how='all', axis=1).fillna(0).to_csv('monthly_stat_by_country_from_2019-10-01_to_2020-11-01.csv')
pandemy_report.keys()
    


Building payload with keywords:  ['Covid', 'Covid19', 'Covid-19', 'Coronavirus', 'quarantine'] 
period:  2019-10-01 2019-11-01
Building payload with keywords:  ['isolation', 'Outbreak', 'muscle aches', 'body aches', 'high temperature'] 
period:  2019-10-01 2019-11-01


KeyboardInterrupt: 

In [7]:
pandemy_stat = pd.DataFrame()
start_of_period = "2019-10-01"
end_of_period = "2019-10-01"
dt_format = "%Y-%m-%d"
def add_day(date, dt_format=dt_format):
    return datetime.strftime((datetime.strptime(date, dt_format) + relativedelta(days=1)), dt_format)
while datetime.strptime(end_of_period, dt_format) < datetime.now():
    full_period = start_of_period + ' ' + end_of_period
    start_of_period, end_of_period = add_day(start_of_period), add_day(end_of_period)
    print(start_of_period, end_of_period)

2019-10-02 2019-10-02
2019-10-03 2019-10-03
2019-10-04 2019-10-04
2019-10-05 2019-10-05
2019-10-06 2019-10-06
2019-10-07 2019-10-07
2019-10-08 2019-10-08
2019-10-09 2019-10-09
2019-10-10 2019-10-10
2019-10-11 2019-10-11
2019-10-12 2019-10-12
2019-10-13 2019-10-13
2019-10-14 2019-10-14
2019-10-15 2019-10-15
2019-10-16 2019-10-16
2019-10-17 2019-10-17
2019-10-18 2019-10-18
2019-10-19 2019-10-19
2019-10-20 2019-10-20
2019-10-21 2019-10-21
2019-10-22 2019-10-22
2019-10-23 2019-10-23
2019-10-24 2019-10-24
2019-10-25 2019-10-25
2019-10-26 2019-10-26
2019-10-27 2019-10-27
2019-10-28 2019-10-28
2019-10-29 2019-10-29
2019-10-30 2019-10-30
2019-10-31 2019-10-31
2019-11-01 2019-11-01
2019-11-02 2019-11-02
2019-11-03 2019-11-03
2019-11-04 2019-11-04
2019-11-05 2019-11-05
2019-11-06 2019-11-06
2019-11-07 2019-11-07
2019-11-08 2019-11-08
2019-11-09 2019-11-09
2019-11-10 2019-11-10
2019-11-11 2019-11-11
2019-11-12 2019-11-12
2019-11-13 2019-11-13
2019-11-14 2019-11-14
2019-11-15 2019-11-15
2019-11-16

In [ ]:
daily_stat_by_countries = pd.DataFrame()
start_of_period = "2019-10-01"
end_of_period = "2019-10-01"
dt_format = "%Y-%m-%d"

def add_day(date, dt_format=dt_format):
    return datetime.strftime((datetime.strptime(date, dt_format) + relativedelta(days=1)), dt_format)

while datetime.strptime(end_of_period, dt_format) < datetime.now():
    full_period = start_of_period + ' ' + end_of_period
    for ISO_code in ISO_codes[:2]:
        diff_stat = pd.DataFrame(execute(covid_keywords, full_period, 'by_region', ISO_code))
        diff_stat["ISO_contry_code"] = ISO_code
        diff_stat["date"] = (datetime.strptime(start_of_period, dt_format))
        daily_stat_by_countries = pd.concat([daily_stat_by_countries, diff_stat])
    start_of_period, end_of_period = add_day(start_of_period), add_day(end_of_period)

Building payload with keywords:  ['Covid', 'Covid19', 'Covid-19', 'Coronavirus', 'quarantine'] 
period:  2019-10-01 2019-10-01
Building payload with keywords:  ['isolation', 'Outbreak', 'muscle aches', 'body aches', 'high temperature'] 
period:  2019-10-01 2019-10-01


In [12]:
daily_stat_by_countries

""


In [101]:
pytrends.get_historical_interest(kw_list, year_start=2020, month_start=1, day_start=1, hour_start=0, year_end=2020, month_end=2, day_end=1, hour_end=0, cat=0, geo='', gprop='', sleep=0)

HTTPSConnectionPool(host='trends.google.com', port=443): Read timed out. (read timeout=5)


,Covid,Coronavirus,quarantine,Fever,headache,isPartial
date,,,,,,
2020-01-01 00:00:00,0,0,1,60,30,False
2020-01-01 01:00:00,0,1,1,61,31,False
2020-01-01 02:00:00,0,0,1,62,31,False
2020-01-01 03:00:00,0,0,0,0,0,False
2020-01-01 04:00:00,0,0,0,0,0,False
...,...,...,...,...,...,...
2020-01-31 20:00:00,0,52,1,3,2,False
2020-01-31 21:00:00,0,62,1,3,2,False
2020-01-31 22:00:00,0,69,1,4,2,False


In [10]:
daily_stat_by_countries.to_csv('daily_stat_by_countries.csv')



In [57]:
pytrends.related_topics()

{}

In [97]:
suggestions_dict = pytrends.suggestions(keyword='Corona')
print(pd.DataFrame(suggestions_dict).drop('mid', axis=1))

                      title     type
0               Coronavirus    Virus
1  Coronavirus disease 2019  Disease
2                    Corona     Beer
3                    Wreath    Topic
4                    corona    Topic


In [13]:
import pandas as pd
pd.read_csv('daily_covid_stat_world.csv').head()

,date,Covid,Covid19,Covid-19,Coronavirus,quarantine,isPartial,isolation,Outbreak,muscle aches,...,casual contact,close contact,direct contact,mild cold symptoms,mild cold,flu-like symptoms,flu-like,pandemic,epidemic,epidemy
0,2019-10-06,0.0,0.0,0.0,0.0,0.0,False,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2019-10-13,0.0,0.0,0.0,0.0,0.0,False,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2019-10-20,0.0,0.0,0.0,0.0,0.0,False,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2019-10-27,0.0,0.0,0.0,0.0,0.0,False,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2019-11-03,0.0,0.0,0.0,0.0,0.0,False,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
pd.read_csv('monthly_stat_by_country_from_2019-10-01_to_2020-11-01.csv')

,geoName,Covid,Covid19,Covid-19,Coronavirus,quarantine,isolation,Outbreak,muscle aches,body aches,...,mild cold symptoms,mild cold,flu-like symptoms,flu-like,pandemic,epidemic,epidemy,period_from,period_to,period
0,Alabama,0.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2019-10-01,2019-11-01,0
1,Alaska,0.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2019-10-01,2019-11-01,0
2,Arizona,0.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2019-10-01,2019-11-01,0
3,Arkansas,0.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2019-10-01,2019-11-01,0
4,California,0.0,0.0,0.0,6.0,94.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2019-10-01,2019-11-01,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5299,Virginia,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,100.0,2020-10-01,2020-11-01,0
5300,Washington,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2020-10-01,2020-11-01,0
5301,West Virginia,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2020-10-01,2020-11-01,0
5302,Wisconsin,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2020-10-01,2020-11-01,0
